<a href="https://colab.research.google.com/github/SanketSarmalkar/Whisper_API_Google_Collab/blob/main/Audio_to_Text_using_Whisper_Large_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
!pip install --q --upgrade pip
!pip install --q --upgrade git+https://github.com/huggingface/transfomers.git accelerate datasets[audio]
!pip install --q flash-attn --no-build-isolation
!pip install transformers --upgrade
!pip install accelerate

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/transfomers.git /tmp/pip-req-build-ttj0gqfs did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/transfomers.git /tmp/pip-req-build-ttj0gqfs did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


# Configuring the CPU/GPU

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
device

'cuda:0'

In [ ]:
torch_dtype

torch.float16

#Configuring the Model

In [ ]:
model_id = "openai/whisper-large-v3"

In [ ]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model= model,
    tokenizer = processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens = 128,
    chunk_length_s = 15,
    batch_size = 16,
    return_timestamps=True,
    torch_dtype = torch_dtype,
    device = device,
)

In [ ]:
pipe

# Hosting API for the Audio to Text Model

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 M

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from fastapi.exceptions import HTTPException
from starlette.status import HTTP_400_BAD_REQUEST
from pydantic import BaseModel
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
@app.post('/predict_text')
async def predict_text(audio: UploadFile = File(...)):
  if audio.content_type != 'audio/mpeg':
        logging.error("Invalid file type")
        raise HTTPException(status_code=HTTP_400_BAD_REQUEST, detail="Invalid file type. Please upload an mp3 file.")
  try:
      audio_bytes = await audio.read()
      logging.info("Successfully read the file")
      result = pipe(audio_bytes)
      return JSONResponse(content={"text": result['text']})
  except Exception as e:
      logging.error(f"Error processing file: {e}")
      raise HTTPException(status_code=HTTP_400_BAD_REQUEST, detail=str(e))

In [ ]:
import os
from google.colab import userdata
from pyngrok import conf
os.environ['NGROK_TOKEN'] = userdata.get("NGROK")
conf.get_default().auth_token = os.environ['NGROK_TOKEN']

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://d72c-34-87-13-243.ngrok-free.app


INFO:     Started server process [1726]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     106.77.35.159:0 - "POST /predict_text/ HTTP/1.1" 307 Temporary Redirect


INFO:     106.77.35.159:0 - "POST /predict_text HTTP/1.1" 400 Bad Request


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


INFO:     106.77.35.159:0 - "POST /predict_text HTTP/1.1" 200 OK
INFO:     106.77.35.159:0 - "POST /predict_text HTTP/1.1" 200 OK
INFO:     106.77.35.159:0 - "POST /predict_text HTTP/1.1" 200 OK


# Doing it Manually

In [ ]:
#audio = "testing_audio_01.mp3"

In [1]:
#result = pipe(audio)

In [ ]:
#result['text']

In [ ]:
#result